In [76]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import mean, min, max


In [2]:
conf = SparkConf().set("spark.ui.port", "4050")
spark = SparkSession.builder.getOrCreate()



In [4]:
spark



In [96]:
df_movies = spark.read.load('/home/peder/dev/big-data/movie-recommendation-system/data/movies_metadata.csv',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

In [97]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(df_movies.count(), len(df_movies.columns)))

The shape of the dataset is 45572 rows by 24 columns


In [98]:
df_movies.filter(col("vote_average").isNull()).count()



498

In [99]:
df_movies = df_movies.na.drop(subset=["vote_average"])


In [100]:
df_movies = df_movies.withColumn("vote_average", df_movies["vote_average"].cast("double"))


In [101]:
df_movies = df_movies.filter((df_movies.vote_average >=0) & (df_movies.vote_average<=10))


In [102]:
df_movies.select(['vote_average']).describe().show()

+-------+------------------+
|summary|      vote_average|
+-------+------------------+
|  count|             41734|
|   mean| 5.590573954904924|
| stddev|1.9515979895067208|
|    min|               0.0|
|    max|              10.0|
+-------+------------------+



In [104]:
result = df_movies.select([mean("vote_average"), min("vote_average"), max("vote_average")])


In [105]:
result.show()

+-----------------+-----------------+-----------------+
|avg(vote_average)|min(vote_average)|max(vote_average)|
+-----------------+-----------------+-----------------+
|5.590573954904924|              0.0|             10.0|
+-----------------+-----------------+-----------------+

